In [107]:
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 0.10.72.1-preview

In [108]:
#!import Skills/CustomSkill/CompanySearchSkill.cs

In [109]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;

In [110]:
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;

In [111]:
IKernel kernel = Kernel.Builder.Build();

In [112]:
kernel.Config.AddAzureOpenAITextCompletionService(
    "GPT3",      
    "text-davinci-003",                  
    "Your Endpoint",    
    "Your OpenAI API Key"          
);

In [113]:
var planner = kernel.ImportSkill(new PlannerSkill(kernel));

In [114]:
planner

key value BucketOutputs Microsoft.SemanticKernel.Orchestration.SKFunction Name BucketOutputs SkillName _GLOBAL_FUNCTIONS_ Description Given an output of a function, parse the output into a number of buckets. IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name input Description The output from a function that needs to be parse into buckets. DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name bucketCount Description The number of buckets. DefaultValue 2 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name bucketLabelPrefix Description The target label prefix for the resulting buckets. Result will have index appended e.g. bucketLabelPrefix='Result' => Result_1, Result_2, Result_3 DefaultValue Result CreatePlan Microsoft.SemanticKernel.Orchestration.SKFunction Name CreatePlan SkillName _GLOBAL_FUNCTIONS_ Description Create a plan using registered functions to accomplish a goal. IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name input Description The goal to accomplish. DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name relevancyThreshold Description The relevancy threshold when filtering registered functions. DefaultValue 2 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name MaxRelevantFunctions Description Limits the number of relevant functions as result of semantic search included in the plan creation request. DefaultValue 100 3 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name excludedFunctions Description A list of functions to exclude from the plan creation request. DefaultValue 4 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name excludedSkills Description A list of skills to exclude from the plan creation request. DefaultValue 5 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name includedFunctions Description A list of functions to include in the plan creation request. DefaultValue ExecutePlan Microsoft.SemanticKernel.Orchestration.SKFunction Name ExecutePlan SkillName _GLOBAL_FUNCTIONS_ Description Execute a plan that uses registered functions to accomplish a goal. IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences Parameters

In [115]:

CompanySearchSkill customSkill = new CompanySearchSkill();

In [116]:
var custom_skill = kernel.ImportSkill (customSkill, "CustomSkill");

In [117]:
var skillsDirectory =  System.IO.Directory.GetCurrentDirectory() + "/Skills";

In [118]:
var write_skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory,"WriteSkill");
var email_skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory,"EmailSkill");

In [119]:
var ask = "查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件";
var plan = await kernel.RunAsync(ask, planner["CreatePlan"]);

In [120]:
plan.Variables.ToPlan().PlanString

<goal>
查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件
</goal>
<plan>
  <function.CustomSkill.WeatherSearch input="Guangzhou" setContextVariable="WEATHER_RESULT"/>
  <function.WriteSkill.Transalte input="$WEATHER_RESULT" setContextVariable="TRANSLATED_RESULT"/>
  <function.WriteSkill.Tips input="$TRANSLATED_RESULT" setContextVariable="TIPS_RESULT"/>
  <function.EmailSkill.WeatherMail input="$TRANSLATED_RESULT;$TIPS_RESULT" setContextVariable="EMAIL_RESULT"/>
  <function._GLOBAL_FUNCTIONS_.BucketOutputs input="$EMAIL_RESULT" bucketCount="2" bucketLabelPrefix="Result" />
</plan>

In [121]:
var executionResults = plan;

In [122]:
int step = 1;
int maxSteps = 30;
while (!executionResults.Variables.ToPlan().IsComplete && step < maxSteps)
{
    var results = await kernel.RunAsync(executionResults.Variables, planner["ExecutePlan"]);
    if (results.Variables.ToPlan().IsSuccessful)
    {
        Console.WriteLine($"Step {step} - Execution results:\n");
        Console.WriteLine(results.Variables.ToPlan().PlanString);

        if (results.Variables.ToPlan().IsComplete)
        {
            Console.WriteLine($"Step {step} - COMPLETE!");
            Console.WriteLine(results.Variables.ToPlan().Result);
            break;
        }
    }
    else
    {
        Console.WriteLine($"Step {step} - Execution failed:");
        Console.WriteLine(results.Variables.ToPlan().Result);
        break;
    }
    
    executionResults = results;
    step++;
    Console.WriteLine("");
}

Step 1 - Execution results:

<goal>
查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件
</goal><plan>
  <function.WriteSkill.Transalte input="$WEATHER_RESULT" setContextVariable="TRANSLATED_RESULT" />
  <function.WriteSkill.Tips input="$TRANSLATED_RESULT" setContextVariable="TIPS_RESULT" />
  <function.EmailSkill.WeatherMail input="$TRANSLATED_RESULT;$TIPS_RESULT" setContextVariable="EMAIL_RESULT" />
  <function._GLOBAL_FUNCTIONS_.BucketOutputs input="$EMAIL_RESULT" bucketCount="2" bucketLabelPrefix="Result" />
</plan>

Step 2 - Execution results:

<goal>
查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件
</goal><plan>
  <function.WriteSkill.Tips input="$TRANSLATED_RESULT" setContextVariable="TIPS_RESULT" />
  <function.EmailSkill.WeatherMail input="$TRANSLATED_RESULT;$TIPS_RESULT" setContextVariable="EMAIL_RESULT" />
  <function._GLOBAL_FUNCTIONS_.BucketOutputs input="$EMAIL_RESULT" bucketCount="2" bucketLabelPrefix="Result" />
</plan>

Step 3 - Execution results:

<goal>
查找广州天气，把结果翻译成中文后根据天气情